In [2]:
#!/usr/bin/env python
### @package dna2aa 
##   Translate the input sequence in the specified or all frames.
##
##   Output: .fa file of translated subject sequences.
##   These sequences are naively translated as internal ORFs,
##   and are likely larger than the sequence's true ORF.
##   The output is best suited to use for hmmer-like alignments.
##
##   Alternatively, using the 'seqbreak' option, ORFs are broken
##   at stop codons.
##
##   Written by Austin Hammond, GSC 2015

import argparse
import sys
import csv

import fastatools as ft

In [ ]:
# TODO
## output ORF sequence itself in addition to peptide
## add eval of complete/partial ORF + peptide

In [2]:
def mtdna2aa(seq):
    tbl = {"ATT":"I", "ATC":"I", "ATA":"M", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
        "TTA":"L", "TTG":"L", "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V", "TTT":"F",
        "TTC":"F", "ATG":"M", "TGT":"C", "TGC":"C", "GCT":"A", "GCC":"A", "GCA":"A",
        "GCG":"A", "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G", "CCT":"P", "CCC":"P",
        "CCA":"P", "CCG":"P", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T", "TCT":"S",
        "TCC":"S", "TCA":"S", "TCG":"S", "AGT":"S", "AGC":"S", "TAT":"Y", "TAC":"Y",
        "TGG":"W", "CAA":"Q", "CAG":"Q", "AAT":"N", "AAC":"N", "CAT":"H", "CAC":"H",
        "GAA":"E", "GAG":"E", "GAT":"D", "GAC":"D", "AAA":"K", "AAG":"K", "CGT":"R",
        "CGC":"R", "CGA":"R", "CGG":"R", "AGA":"-", "AGG":"-", "TAA":"-", "TAG":"-",
        "TGA":"W"}
    j = 0 # for iteration
    aa = ""
    seql = len(seq)
    if seql % 3 != 0:
        seq = seq[0:(seql - seql%3)] #trim to multiple of 3, but should be already
    seql = len(seq) #recalculate
    for j in range(0, seql, 3):
        j += 1
        if j == (seql - 1):
            codon = seq[j:]
        else:
            codon = seq[j-1:j+2]
    # print codon
        aa = aa + tbl[codon]
    # force first codon to be M
    aa = "M" + aa[1:]

    return aa

In [3]:
def mtsixframe(seq):
    one = mtdna2aa(seq)
    two = mtdna2aa(seq[1:])
    three = mtdna2aa(seq[2:])
    none = mtdna2aa(ft.revcomp(seq))
    ntwo = mtdna2aa(ft.revcomp(seq)[1:])
    nthree = mtdna2aa(ft.revcomp(seq)[2:])

    return [one, two, three, none, ntwo, nthree]

In [123]:
def trim_met(sqn):
    # trim the given sequence to the first methionine (M) residue
    # if no M, then 'no_met' is returned
    for pos in range(0, len(sqn)):
        res = sqn[pos]
        if res is not "M":
            continue
        else:
            outsqn = sqn[pos:]
            return outsqn
    else:
        return "no_met"

In [167]:
def trim_off_stop(sqn):
    # trim off the stop codon marks that I introduce (-)
    # only trims "-" at end of the peptide
    if "-" not in sqn:
        return sqn
    rev = sqn[::-1]
    for pos in range(0, len(rev)):
        res = rev[pos]
        if res is "-":
            continue
        else:
            trim = rev[pos:][::-1]
            break
    return trim

In [152]:
def len_thresh(sqn, threshold):
    # drop sequences under a given length
    # sequences where len >= threshold will be output
    # sqn is a single translated sequence [STR]
    # threshold = [INT]
    # if len(sqn) < threshold, 'short_pep_LEN_residues' returned
    thresh = int(threshold) # just in case...
    sqnS = sqn.strip() # avoid trailing whitespace, newlines
    sqnL = len(sqnS)
    if sqnL >= threshold:
        return sqnS
    else:
        msg = "".join(["short_pep_", str(sqnL), "_residues"])
        return msg

In [ ]:
# setup parser
parser = argparse.ArgumentParser(
    description="Naively translate the input DNA sequence.")
parser.add_argument(
    "--sequence", "-s", action="store", help="The fasta file of DNA sequences [STDIN]",
    default="stdin")
parser.add_argument("--frame", "-f", action="store",
    choices=["+1","+2","+3","-1","-2","-3","all"],
    help="Frame to use for sequence translation [all]", default="all")
parser.add_argument("--seqbreak","-b",action="store_true",
    help="Break translations at in-frame stop codons")
parser.add_argument("--outfile", "-o", action="store",
    help="Name to use for output fasta file [STDOUT]", default="stdout")
parser.add_argument("--mito", "-m", action="store_true", 
    help=("Sequence is mitochondrial; use vertebrate Mt code."
          " Does not allow stop codons to be completed by polyadenylation."))
parser.add_argument("--min_length", "-l", help="Minimum peptide length to output [30]",
                    action="store", type=int, default=30)
parser.add_argument("--trim_to_met", "-t", help="Trim peptides to first M",
                    action="store_true")

In [183]:
### FOR DEV ###
frm = "all"
outfile = "stdout"
#outfile = "tests/test-out.txt"
fasta = "tests/untitled.txt"
fastadic = {}
outdic = {}
seqbreak = False
mito = False
#frm = "+1"
trim = True
minlen=5
### END DEV ###

In [ ]:
# parse args
args = parser.parse_args()
frm = args.frame
fasta = args.sequence
fastadic = {}
outfile = args.outfile
outdic = {}
mito = args.mito
seqbreak = args.seqbreak
minlen = args.min_length
trim = args.trim_to_met

In [184]:
# check that the frame was passed properly
if frm == "all":
    pass
elif "+" in frm:
    pass
elif frm == "1":
    frm = "+1"
elif frm == "2":
    frm = "+2"
elif frm == "3":
    frm = "+3"
elif "-" not in frm:
    print "ERROR: I didn't understand what reading frame you want."
    print "Please specify 'all' or a single frame including + or - and retry."
    sys.exit(1)

In [185]:
# define possible reading frames
if frm == "all":
    frames = ["+1","+2","+3","-1","-2","-3"]
else:
    frames = [frm]

In [186]:
# read in sequence input
if fasta == "stdin":
    for rec in ft.fasta_iter(sys.stdin):
        fastadic[rec[0].split(" ")[0]] = rec[1]

else:
    with open(fasta) as fa:
        for rec in ft.fasta_iter(fa):
            sid = rec[0].split(" ")[0]
            sqn = rec[1]
            fastadic[sid] = sqn

In [187]:
# perform translations
if seqbreak:
    # setup separate dict to hold split translations
    transdic = {}

for sid, sqn in fastadic.items():
    if mito == True:
        if frm == "all":
            trns = mtsixframe(sqn)
        elif frm == "+1":
            trns = mtdna2aa(sqn)
        elif frm == "+2":
            trns = mtdna2aa(sqn[1:])
        elif frm == "+3":
            trns = mtdna2aa(sqn[2:])
        elif frm == "-1":
            trns = mtdna2aa(ft.revcomp(sqn))
        elif frm == "-2":
            trns = mtdna2aa(ft.revcomp(sqn)[1:])
        else:
            trns = mtdna2aa(ft.revcomp(sqn)[2:])

    else:
        if frm == "all":
            trns = ft.sixframe(sqn)
        elif frm == "+1":
            trns = ft.dna2aa(sqn)
        elif frm == "+2":
            trns = ft.dna2aa(sqn[1:])
        elif frm == "+3":
            trns = ft.dna2aa(sqn[2:])
        elif frm == "-1":
            trns = ft.dna2aa(ft.revcomp(sqn))
        elif frm == "-2":
            trns = ft.dna2aa(ft.revcomp(sqn)[1:])
        else:
            trns = ft.dna2aa(ft.revcomp(sqn)[2:])

    # setup output dict entry
    outdic[sid] = {}
    if seqbreak:
        # if frm == all, trns is a list; otherwise is a string #TODO make always a list
        if type(trns) == list:
            if frm == "all":
                transzip = zip(frames, trns)
            else:
                transzip = (frm, trns)
            for rec in transzip:
                thisframe = rec[0]
                thisseq = rec[1]
                #splitsies = [x.split("-") for x in thisseq]
                splitsies = thisseq.split("-")
                # drop empty frames
                splitsies = [x for x in splitsies if x]
                outdic[sid][thisframe] = splitsies
        else:
            splitsies = trns.split("-") # split into ORFs
            multi = [x for x in splitsies if x] # drop empty ORFs
            outdic[sid][frm] = multi
    else:
        if type(trns) == list:
            outdic[sid] = {}
            if frm == "all":
                transzip = zip(frames, trns)
            else:
                transzip = (frm, trns)
            for rec in transzip:
                thisframe = rec[0]
                thisseq = rec[1]
                outdic[sid][thisframe] = thisseq
        else:
            outdic[sid][frm] = trns

In [188]:
### FOR DEV ###
print "frm", frm
print"frames", frames
print "fastadic", fastadic
print "trns", trns
print "transzip", transzip
print "splitsies", splitsies, len(splitsies)
print "outdic", outdic
print trim_met(outdic["seq2"]["+1"])
print len_thresh(trim_met(transzip[4][1]), 2)
### END DEV ###

frm all
frames ['+1', '+2', '+3', '-1', '-2', '-3']
fastadic {'seq3': 'TGGATGACACTAGGCATATAA', 'seq2': 'TGGATTACACTAGGCATATAA', 'seq1': 'ATGGATTACACTAGGCATTAA', 'seq5': 'ATGGAACATTACTAATAATAA', 'seq4': 'TGGATGACACTAGGCATATAAAAT'}
trns ['WMTLGI-N', 'G-H-AYK', 'DDTRHIK', 'ILYA-CHP', 'FYMPSVI', 'FICLVSS']
transzip [('+1', 'WMTLGI-N'), ('+2', 'G-H-AYK'), ('+3', 'DDTRHIK'), ('-1', 'ILYA-CHP'), ('-2', 'FYMPSVI'), ('-3', 'FICLVSS')]
splitsies ['FICLVSS'] 1
outdic {'seq3': {'+3': 'DDTRHI', '+2': 'G-H-AY', '+1': 'WMTLGI-', '-1': 'LYA-CHP', '-3': 'ICLVSS', '-2': 'YMPSVI'}, 'seq2': {'+3': 'DYTRHI', '+2': 'GLH-AY', '+1': 'WITLGI-', '-1': 'LYA-CNP', '-3': 'ICLV-S', '-2': 'YMPSVI'}, 'seq1': {'+3': 'GLH-AL', '+2': 'WITLGI', '+1': 'MDYTRH-', '-1': 'LMPSVIH', '-3': 'NA-CNP', '-2': '-CLV-S'}, 'seq5': {'+3': 'GTLLII', '+2': 'WNITNN', '+1': 'MEHY---', '-1': 'LLLVMFH', '-3': 'IISNVP', '-2': 'YY--CS'}, 'seq4': {'+3': 'DDTRHIK', '+2': 'G-H-AYK', '+1': 'WMTLGI-N', '-1': 'ILYA-CHP', '-3': 'FICLVSS', '-2': 'FYM

In [190]:
# write the translations to a file
# TODO clean up this redundant code
if outfile == "stdout":
    for nam, trns in outdic.items():
        if seqbreak:
            for frame in frames:
                seqno = 0
                for rec in trns[frame]:
                    # get rid of trailing "-"
                    pep = trim_off_stop(trns[frame][seqno])
                    # apply trim_met before length cutoff
                    if trim:
                        trimmed = trim_met(pep)
                        if "no_met" in trimmed:
                            continue
                        else:
                            # check peptide length >= minlen
                            if "short_pep" not in len_thresh(trimmed, minlen):
                                header = "".join([">", nam, "_frame=", frame,
                                                  ";ORF_num=", str(seqno + 1),
                                                  "\n"])
                                print header, trimmed
                                seqno += 1
                            else:
                                continue
                    else:
                        # check peptide length >= minlen
                        if "short_pep" not in len_thresh(pep, minlen):
                            header = "".join([">", nam, "_frame=", frame,
                                              ";ORF_num=", str(seqno + 1),
                                              "\n"])
                            print header, pep
                            seqno += 1
                        else:
                            continue
        else:
            for frame in frames:
                # get rid of trailing "-"
                pep = trim_off_stop(trns[frame])
                if trim:
                    trimmed = trim_met(pep)
                    if "no_met" in trimmed:
                        continue
                    else:
                        if "short_pep" not in len_thresh(trimmed, minlen):
                            print "".join([">", nam, "_frame=", frame, "\n", trimmed])
                        else:
                            continue
                else:
                    if "short_pep" not in len_thresh(pep, minlen):
                        print "".join([">", nam, "_frame=", frame, "\n", pep])
                    else:
                        continue
else:
    with open(outfile,"w") as finalout:
        for nam, trns in outdic.items():
            if seqbreak:
                for frame in frames:
                    seqno = 0
                    for rec in trns[frame]:
                        pep = trim_off_stop(trns[frame][seqno])
                        if trim:
                            trimmed = trim_met(pep)
                            if "no_met" in trimmed:
                                continue
                            else:
                                # check peptide length >= minlen
                                if "short_pep" not in len_thresh(trimmed, minlen):
                                    header = "".join([">", nam, "_frame=", frame,
                                                      ";ORF_num=", str(seqno + 1),
                                                      "\n"])
                                    print header, trimmed
                                    seqno += 1
                                else:
                                    continue
                        else:
                            # check peptide length >= minlen
                            if "short_pep" not in len_thresh(pep, minlen):
                                header = "".join([">", nam, "_frame=", frame,
                                                  ";ORF_num=", str(seqno + 1), "\n"])
                                print >> finalout, header, pep
                                seqno += 1
                            else:
                                continue
            else:
                for frame in frames:
                    pep = trim_off_stop(trns[frame][seqno])
                    if trim:
                        trimmed = trim_met(pep)
                        if "no_met" in trimmed:
                            continue
                        else:
                            # check peptide length >= minlen
                            if "short_pep" not in len_thresh(trimmed, minlen):
                                print >> finalout, "".join([">", nam, "_frame=",
                                                            frame, "\n", trimmed])
                            else:
                                continue
                    else:
                        # check peptide length >= minlen
                        if "short_pep" not in len_thresh(pep, minlen):
                            print >> finalout, "".join([">", nam, "_frame=", frame,
                                                        "\n", pep])
                        else:
                            continue

### EOF ###

>seq3_frame=+1
MTLGI
>seq3_frame=-2
MPSVI
>seq2_frame=-2
MPSVI
>seq1_frame=+1
MDYTRH
>seq1_frame=-1
MPSVIH
>seq4_frame=+1
MTLGI-N
>seq4_frame=-2
MPSVI
